In [1]:
from pantheon import pantheon
from pantheon.utils import exceptions as exc
import asyncio

#Indicate which server you want you matches form
server = "na1"
#Use your own RIot API key, find more about here : https://developer.riotgames.com/
api_key = ""

panth = pantheon.Pantheon(server, api_key, True)

In [2]:
async def getAllMatchId(accountId):
    matches = []
    beginIndex = 0
    while True:
        try:
            matchlist = await panth.getMatchlist(accountId, {"beginIndex":beginIndex,"queue":"470","season":"11"}) #Params for matchlist here filter for games played in 3v3 Flex and for season 8 only
            matches += [g['gameId'] for g in matchlist['matches']]
            if len(matchlist['matches']) == 100:
                beginIndex += 100
            else:
                break
        except exc.NotFound as e:
            break
        except (exc.RateLimit, exc.ServerError, exc.Timeout) as e:
            print(e)
        except Exception as e:
            print(e)
    return matches

async def getChallengersAccountIds(queue):
    try:
        challResponse = await panth.getChallengerLeague(queue)
        summIds = [e["playerOrTeamId"] for e in challResponse["entries"]]
        tasks = [panth.getSummoner(summId) for summId in summIds]
        summResponses = await asyncio.gather(*tasks)
        accountIds = [i['accountId'] for i in summResponses]
        return accountIds
    except Exception as e:
        print(e)

async def getMatchIdsFromMultipleAccounts(accountIds):
    return await asyncio.gather(*[getAllMatchId(accountId) for accountId in accountIds])

In [3]:
loop = asyncio.get_event_loop()  

challAccountIds = loop.run_until_complete(getChallengersAccountIds("RANKED_FLEX_TT"))

matches = loop.run_until_complete(getMatchIdsFromMultipleAccounts(challAccountIds))

matchesList = [a for b in matches for a in b]

In [ ]:
import pymongo
client = pymongo.MongoClient()

#Select the right mongo DB and table
db = client.game
mongoTable = db["gameTT2_NA"]


mongoTable.create_index([("gameId" , pymongo.ASCENDING),("platformId", pymongo.ASCENDING)],name="gameId_index", unique=True, sparse=True)

In [9]:
async def getMatch(matchId):
    try:
        data, timeline = await asyncio.gather(
            panth.getMatch(matchId),
            panth.getTimeline(matchId)
        )

        data['timeline'] = timeline

        try:
            mongoTable.insert_one(data);
        except pymongo.errors.DuplicateKeyError:
            pass
    except Exception as e:
        print(e)
        print(matchId)

In [ ]:
output = loop.run_until_complete(asyncio.gather(*[getMatch(matchId) for matchId in set(matchesList)]))